# Chapter 2

-------------------------------------------------------------------------------------

## Cunstom sequence 생성

1. range로 indexing하는 결과는 해당 class와 같은 type의 instance여야 한다.
2. slice에 의해 제공된 범위는 python이 하는 것처럼 마지막 요소는 제외해야한다.

첫 번째 규칙을 들어보자.
예를 들어 아래와 같은 경우가 있다.

In [2]:
range(100)[25:50]

range(25, 50)

- *range를 slicing하면 range를 return한다.!!*

두 번째 규칙은 일관성에 관한 이야기이다.<br>
(마지막 요소는 포함하지 않는다)<br>

## Context manager

---------------------------------------------------------------------------------------------------------

- 이 기능은 주요한 동작 전후에 어떤 작업을 실행할 때 유용하다.

예를 들어 file 을 여는 코드가 있다. 아래 두 코드는 정확히 같은 동작을 한다.

In [4]:
! echo hello foo > foo.txt

fd = open('foo.txt')
try:
    pass
finally:
    fd.close()

In [5]:
with open('foo.txt') as fd:
    pass

아래의 코드가 훨씬 *Pythonic* 하며 훨씬 이해하기 쉬운 코드이다.

## Context manager의 구현

--------------------------------------------------------------------------------------------------------

### enter, exit

- 아래의 두 magit method를 구현함으로써 context manager를 구현할 수 있다.   
   
    \_\_enter__  : with문은 해당 magic method를 호출하고 return value를 as 뒤에 붙는 변수에 할당한다. <br>
    \_\_exit__   : 모든 context들이 실행되고 나고 해당 블록이 종료될 때 이 magit method가 실행된다.

    \_\_exit__ 은 exception이 발생한 경우 exception을 parameter로 받기 때문에 예외를 처리하기에도 좋다.
  
### contextlib module 이용

아래와 같이 구현 가능하다.

In [ ]:
! pip3 install contextlib

In [ ]:
import contextlib

# @contextlib.contextmanager
# def db_handler():
    # stop_database()
    # yield
    # start_database()

위와 같이 구현이 가능하다. <br>

여기서 중요한 점은 decorator로 context manager를 정해주려면 <br>
- **해당 함수가 generator 여야한다.**

# Properties, attributes, other types of object method

------------------------------------------------------------------------------

## underscore in python

- python에서의 모든 property와 function들은 public이다.
- underscore(_)로 시작하는 attribute은 해당 객체에 대해 private을 의미하며<br>
  외부에서 호출하지 않아줬으면 하는 권고이다.

In [3]:
class Connector:
    def __init__(self, source):
        self.source = source
        self._timeout = 60

conn = Connector("postgrespl://localhost")
print(conn.source)
print(conn._timeout)
print(conn.__dict__)

postgrespl://localhost
60
{'source': 'postgrespl://localhost', '_timeout': 60}


위에서 보듯이 둘 다 모두 접근할 수 있다.

하지만 아래와 같이 __timeout으로 다시 정의해보자

In [5]:
class Connector2:
    def __init__(self, source):
        self.source = source
        self.__timeout = 60

conn = Connector2("postgrespl://localhost")
conn.__timeout

AttributeError: 'Connector2' object has no attribute '__timeout'

위와 같은 결과를 얻는다 분명 __time attribute를 정의했는데 찾을 수 없다고한다.<br>
하지만 아래와 __dict__를 호출해 attribute를 다시 호출해보자

In [7]:
conn.__dict__

{'source': 'postgrespl://localhost', '_Connector2__timeout': 60}

- \_\_timeout이 아니다 \_Connector2__timeout으로 이름이 바뀌었다..!<br>
  python에서는 밑줄이 두 개인 attribute에 대하여 처음의 이름과 다른 이름을 부여한다.

- 이를 'name-mangling' 이라고 한다.
- 여러번 확장되는 클래스의 method 이름 충돌 없이 overriding 하기 위함이다.<br>

## Property

- java, C++에서는 getter, setter를 따로 정의 하지만 python 에서는 property를 사용한다.<br>

아래의 예제를 살펴보자.

In [23]:
import re

EMAIL_FORMAT = re.compile(r"[^@]+@[^@]+[^@]+")

def is_valid_email(potentially_valid_email: str):
    return re.match(EMAIL_FORMAT, potentially_valid_email) is not None

class User:
    def __init__(self, username):
        self.username = username
        self._email = None

    @property
    def email(self):
        return self._email

    @email.setter
    def email(self, new_email):
        if not is_valid_email(new_email):
            raise ValueError(f'Invalid email : {new_email}')
        self._email = new_email

- Q : 왜 class 밖에서 property를 정의하지?
- A : 책 오류임 class 안에 넣어야 동작함

- 하나의 method에서 한 가지의 일을 해야해

# Iterable Object

--------------------

- Iterable : \_\_iter__ magic method를 구현한 obj
- Iterator : \_\_next__ magic method를 구현한 obj

- python의
 반복은 iterable protocol에 의해 작동한다. 이는 아래와 같은 검사를 한다.<br>

1. \_\_next__, \_\_iter__ 중 하나를 갖고 있는지?
2. obj이 sequence이고, \_\_len__, __getitem__을 모두 가졌는지?

위와 같이 iterable한 obj을 만드는 방법에는 두 가지가 있다.

## Create iterable

- obj을 반복할 때 iter()를 호출한다. 아래와 같이 코드를 작성할 수 있다.

In [59]:
from datetime import timedelta
from datetime import date

class DataRangeIterable:
    """
        자체 iterator method (__next__()) 를 갖는 Iterable
    """

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

for day in DataRangeIterable(date(2019, 1, 1), date(2019, 3, 1)):
    print(day)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28


- python에서 for문을 돌린다는 것은 next()함수를 StopIteration이 발생할 때까지<br>
  호출한다는 것을 알 수 있다.

- for문의 작동 순서

###for day in DateRangeIterable: 을 실행했다고 하자
1. check DateRangeIterable.\_\_iter\_\_()
2. 그러면, iter(__iter__(self))
3. __iter__(self) -> *self* 이므로, next(*self*)
4. while not StopIteration

그러나 위와 같이 iterable을 작성한다면 _present_day가 end_date가 될 때<br>
더 이상 반복하지 못한다는 단점이 있다.

### Contatiner iterabel

- \_\_iter__ 에서 generate를 사용하는 방법이 있다.

